In [8]:
# Neccessary Imports

import pandas as pd
import yfinance as yf
import backtesting 
from backtesting import Backtest, Strategy
from backtesting.lib import resample_apply, crossover
import matplotlib.pyplot as plt
import ta
pd.options.mode.chained_assignment = None

In [9]:
class TradeStrategy(Strategy):
    def init(self):
        close = self.data.Close
        high = self.data.High
        low = self.data.Low
        
        self.stoch_k = self.I(ta.momentum.stochrsi_k, pd.Series(close))
        self.stoch_d = self.I(ta.momentum.stochrsi_d, pd.Series(close))
        
        # Three ATR method
        
        self.EMA_8 = self.I(ta.trend.ema_indicator, pd.Series(close), window=8)
        self.EMA_14 = self.I(ta.trend.ema_indicator, pd.Series(close), window=14)
        self.EMA_50 = self.I(ta.trend.ema_indicator, pd.Series(close), window=50)
        
        self.atr = self.I(ta.volatility.average_true_range,pd.Series(high), pd.Series(low),pd.Series(close))
        
        
    def next(self):
        price = self.data.Close
        
        if (crossover(self.stoch_k, self.stoch_d) and price > self.EMA_8 and
            self.EMA_8 > self.EMA_14 and self.EMA_14 > self.EMA_50):
            self.buy()
        
        
        

In [10]:
# bt = Backtest(df, TradeStrategy,cash=10000, commission=.002,
#               exclusive_orders=True)
# stats = bt.run()
import os
results = []

path = 'train/daily/' # Specify file path

files = [i for i in os.listdir(path) if '.csv' in i]
for file in files:
    df = pd.read_csv(path+file,index_col=0)
    df.rename(columns={'close':'Close', 'open':'Open', 'high':'High', 'low':'Low', 'volume':'Volume'}, inplace=True)
    df.index = pd.to_datetime(df.index)
    
    bt = Backtest(df, TradeStrategy,cash=100000, commission=.002,
              exclusive_orders=True)
    stats = bt.run()
    results.append([stats,bt])


In [11]:
# You can see the results by results[i][0]
results[0][0]
# To check other stats, simply provide i value to results[i][0]

# And to visualize the data just put one in the second argument , eg: results[i][1].plot()
results[0][1].plot() # Use i indicator to see all the results and visualizations.

Row(id='2794', ...)